# TidyData: Column: validate=

The following is an introduction to basic validation in datachef and usage of the `validate=` keyword provided by the `Column` class when constructing `TidyData`.

| <span style="color:green">Note - this keyword uses exactly the same `against` module we used when explaining `selection.validate()` method earlier in this documentation - the difference here is intent (explained below in more detail).</span>|
|-----------------------------------------|

## validate= vs .validate()

The key difference to understand is as follow:

- The `<selectable>.validate()` method run against **selections** so police your _extraction logic_.
- Validations via the `Column(validation=)` keyword runs against the **output** so police your _final product_.

Consider the following scenarios:

- 1.) You want to select an "anchor cell" or a selection of cells for the sole purpose of subtracting it from another selection. It could be important to confirm these selections are accurate but because they're not directly extracted values then `validation=` will never see them (just the consequence of them) so the `<selectable>.validate()` is more appropriate.

- 2.) You're are using `apply=` to cleanse cell value data _at the point of extraction_ and need to make sure the correct things are happening, the `<selectable>.validate()` method will **never see these cleansed values**, but `validation=` will.

There are nuances on where its best to use each but the pithy version is "use both strategies, wherever possible and as much as its practical to do so".

## Source Data

The data source we're using for these examples is shown below:

| <span style="color:green">Note - this particular table has some very verbose headers we don't care about, so we'll be using `bounded=` to remove them from the previews as well as to show just the subset of data we're working with.</span>|
|-----------------------------------------|

The [full data source can be downloaded here](https://github.com/mikeAdamss/datachef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx). We'll be using th 10th tab named "Table 3c".

In [28]:
from typing import List
from datachef import acquire, preview
from datachef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://github.com/mikeAdamss/datachef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx")
preview(tables[9], bounded="A4:H10")

,A,B,C,D,E,F,G,H
4,Percentage change 3 months on previous 3 months,,,,,,,
5,Time period,Public new housing,Private new housing,Total new housing,Infrastructure new work,Public other new work,Private industrial new work,Private commercial new work
6,Dataset identifier code,MVO6,MVO7,MVO8,MVO9,MVP2,MVP3,MVP4
7,Jun 2010,5.6,9.8,8.8,3,4.3,3.7,1.9
8,Jul 2010,2,5.6,4.8,0.2,-0.2,9.7,3.5
9,Aug 2010,5.5,4.5,4.7,-2.9,-2.9,24.4,5.9
10,Sep 2010,11.7,7.5,8.5,-6.8,-3.3,16.1,5.3


## Simple Regex Validation

For this example we're going to use the same `against` module we used early in this documentation. This has a simple regex validator that works in exactly the way explained above.

i.e `against` is just a convenience, you could just define this yourself.

So the following example is valid for the regex provided:

In [29]:
from typing import List
from datachef import acquire, against
from datachef.direction import right, down
from datachef.output import TidyData, Column
from datachef.selection import XlsxSelectable

tables: List[XlsxSelectable] = acquire.xlsx.http("https://github.com/mikeAdamss/datachef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx")
table = tables[9]

observations = table.excel_ref("B7:H10").label_as("Observations")
dataset_identifier_code = table.excel_ref("B6").expand(right).label_as("Dataset Identifier Codes")

# Note: matches a regex of capital M followed by anything
tidy_data = TidyData(
    observations,
    Column(dataset_identifier_code.finds_observations_directly(down), validate=against.regex("M.*"))
)

## A Note on Lazy Evaluation

One thing you many notice about the above is that the validation error does not occur until we try and print the `tidy_data` variable, this is because the `TidyData` class uses _lazy evaluation_.

Simply put, this means the tidy data is never extracted until the last possible moment that is has to be, in this case when the users goes to print the results.